In [1]:
#!/usr/bin/env python
from __future__ import print_function

import actionlib
import rospy

from control_msgs.msg import PointHeadAction, PointHeadGoal

from geometry_msgs.msg import PointStamped

from people_msgs.msg import PositionMeasurementArray, PositionMeasurement
from tf import TransformListener

In [2]:
faces_topic = "face_detector/people_tracker_measurements_array"
face_tracked_topic  = "face_tracked"
action_client = "head_controller/point_head"

In [3]:
class Follower():

    def __init__(self, *args):
        self.tf = TransformListener()
        self.client = actionlib.SimpleActionClient(action_client, PointHeadAction)
        self.client.wait_for_server()
        rospy.Subscriber(faces_topic, PositionMeasurementArray, self.callback)
        self.person_tracked_pub = rospy.Publisher(face_tracked_topic, PositionMeasurement, queue_size=10)
        self.person_tracked = PositionMeasurement()
        self.rate = rospy.Rate(30)
        rospy.spin()

    def callback(self, msg):
        for person in msg.people:
            if person.object_id == self.person_tracked.object_id:
                break
        point = PointStamped()
        point.header = person.header
        point.point = person.pos
        self.set_point_head(point)
        self.person_tracked_pub.publish(person)
        self.person_tracked = person
        self.rate.sleep()


    def set_point_head(self, point):
        goal = PointHeadGoal()
        corrected_point = self.tf.transformPoint("base_link", point)
        goal.target = corrected_point
        goal.min_duration = rospy.Duration(.5)
        goal.max_velocity = 1

        self.client.send_goal(goal)
#         self.client.wait_for_result()
#         return self.client.get_result()

In [ ]:
if __name__ == '__main__':
    rospy.init_node('looker')
    try:
        ne = Follower()
    except rospy.ROSInterruptException:
        pass